# Setup API keys
Since we're going to use openAI chat backends in this tutorial, we only need to setup the enviornment variable OPENAI_API_KEY.
If you don't have an openAI API account. You can register one create your own key at `platform.openai.com`.

In [1]:
import sys
import os
from dotenv import load_dotenv
load_dotenv()
CHATARENA_PATH = os.getenv("CHATARENA_PATH")
sys.path.append(CHATARENA_PATH)



In [2]:
import cohere
from chatarena.backends import CohereAIChat

client = cohere.Client(os.getenv("COHEREAI_API_KEY"))
backend = CohereAIChat()

from chatarena.agent import Player

players = ["Nancy", "Tom", "Cindy", "Jack", "Rose", "Edward"]


role_description = """
Now you are playing the game called "who is the spy".

There are 4 players in this game. 
At the beginning of the game, everyone will receive a word.
There is one spy who will receive a spy word, while others will receive a common word.
Spy word is different but relevant to common words. For example, the spy word can be "apple", and the common word is "banana".

There are two stages in each round of the game.

The first stage is describing stage:
Everyone needs to use a word or a few words to say something in turn about the word he received without directly saying the word.
The funniest part of the game is that since you do not know other's words, you are not sure whether you are the spy.
So, you can only infer who have the different based on other players description.

The second stage is the voting stage:
After everyone speaks, each player must vote for a player who he thinks is the spy and tell others why. Players can not repeat other players' speeches or their own speeches in the previous round. The player getting the most votes will be eliminated from the game.

If the spy is eliminated, the rest of the players win.
If there are less than three players, and the spy still lives, then the spy wins.
"""

format_specification = """
Your output should be format in a json with the following schema:
```
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "type": "object",
  "properties": {
    "word": {
      "description": "your secret word",
      "type": "string"
    },
    "arguments": {
      "description": "what you are going to say to your opponent",
      "type": "string"
    }
  },
  "required": ["word", "arguments"]
}
```

For example:
```
{\n  "word": "banana",\n  "arguments": "Is it a fruit that monkeys love." \n}

```
"""

player_0 = Player(name=players[0],
               role_desc=role_description+format_specification,
               backend=backend)

player_1 = Player(name=players[1],
               role_desc=role_description+format_specification,
               backend=backend)

player_2 = Player(name=players[2],
               role_desc=role_description+format_specification,
               backend=backend)

player_3 = Player(name=players[3],
               role_desc=role_description+format_specification,
               backend=backend)

player_4 = Player(name=players[4],
               role_desc=role_description+format_specification,
               backend=backend)

player_5 = Player(name=players[5],
               role_desc=role_description+format_specification,
               backend=backend)


Now let's test the buyer with a mock message.

In [3]:
from chatarena.message import Message

mock_message = Message(
    agent_name="Moderator",
    content=f"{players[0]}, now describe your word",
    turn=1,
)

response = player_0.act([mock_message])
response

'```json\n{\n  "word": "sunflower",\n  "arguments": "This big yellow flower with a cheerful face always points towards the sun and brightens up the day."\n}\n```'

# Building the Enviornment
With players ready, let's build the enviornment now.
In general, the interface design of ChatArena enviornment follows the gym/pettingzoo abstraction.
Like gym envionrment, you need to implement `reset` and `step` methods.
`reset` is called when an episode is over or early terminated which reinitialize the whole game.
`step` drives the dynamics of the game, which takes the action of a single agent as input do the transitions and return the observation, reward and terminal signal.

Besides basic gym interface, you also have to implement `get_observation` and `get_next_speaker` for a chatArena enviornment.

For this game in particular:



In [4]:
from chatarena.arena import Arena
from chatarena.environments.spyfall import SpyFall

players = ["Nancy", "Tom", "Cindy", "Jack", "Rose", "Edward"]

env = SpyFall(player_names=players)
arena = Arena([player_0, player_1, player_2, player_3, player_4, player_5], env)
arena.launch_cli(interactive=False)

arena.save_history("test1.json")

_________  .__               __      _____
\_   ___ \ |  |__  _____   _/  |_   /  _  \  _______   ____    ____  _____
/    \  \/ |  |  \ \__  \  \   __\ /  /_\  \ \_  __ \W/ __ \  /    \ \__  \
\     \____|   Y  \ / __ \_ |  |  /    |    \ |  | \/\  ___/ |   |  \ / __ \_
 \______  /|___|  /(____  / |__|  \____|__  / |__|    \___  >|___|  /(____  /
        \/      \/      \/                \/              \/      \/      \/

🏟 Chat Arena Initialized!

Environment (spyfall) description:
None

[Nancy (cohere-chat)] Role Description:

Now you are playing the game called "who is the spy".

There are 4 players in this game. 
At the beginning of the game, everyone will receive a word.
There is one spy who will receive a spy word, while others will receive a common word.
Spy word is different but relevant to common words. For example, the spy word can be "apple", and the common word 
is "banana".

There are two stages in each round of the game.

The first stage is describing stage:
Everyone needs to use a word or a few words to say something in turn about the word he received without directly 
saying the word.
The funniest part of the game is that since you do not know other's words, you are not sure whether you are the 
spy.
So, you can only infer who have the different based on other players description.

The second stage is the voting stage:
After everyone speaks, each player must vote for a player who he thinks is the spy and tell others why. Players can
not repeat other players' speeches or their own speeches in the previous round. The player getting the most votes 
will be eliminated from the game.

If the spy is eliminated, the rest of the players win.
If there are less than three players, and the spy still lives, then the spy wins.

Your output should be format in a json with the following schema:
```
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "type": "object",
  "properties": {
    "word": {
      "description": "your secret word",
      "type": "string"
    },
    "arguments": {
      "description": "what you are going to say to your opponent",
      "type": "string"
    }
  },
  "required": ["word", "arguments"]
}
```

For example:
```
{
  "word": "banana",
  "arguments": "Is it a fruit that monkeys love." 
}

```

...

[Tom->all]: Host: Great, now we hear from Bob.

[Cindy->all]: ```json
{
  "word": "Cantaloupe",
  "arguments": "Think of a juicy orange surprise, a summer favorite that's sweet and delicious, with a hint of 
sunshine!"
}
```

[Jack->all]: Oh, Cindy, you have the same word as Bob! That makes things interesting. 

```json
{
  "word": "Orange",
  "arguments": "A citrusy surprise awaits, a sweet and tangy delight with a bright color, perfect for a refreshing 
break!"
}
```

[Rose->all]: Host: Alright! Jack, you're the spy! Everyone else had the word "orange", while you had "watermelon". 

Now, the voting stage begins. Players, vote for who you think is the spy, and tell us why. 

Nancy, you're up first.

========= Arena Ended! ==========